Using Azure OpenAI Search

In [4]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.0.1-alpha"
#r "nuget: System.Linq.Async, 6.0.1"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.0.1-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha System.Linq.Async, 6.0.1

In [5]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Connectors.AzureAISearch;

#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0021
var memory = new MemoryBuilder()
    .WithAzureOpenAITextEmbeddingGeneration("mrtextembeddingada002", azureEndpoint, apiKey)
    .WithMemoryStore(new AzureAISearchMemoryStore("https://mrsearch.search.windows.net", Environment.GetEnvironmentVariable("AZURE_OPENAI_SEARCH_QUERYKEY")))
    .Build();

In [6]:
var questions = new[]
{
    "Application roles sample"
};
foreach (var q in questions)
{
    var response = await memory.SearchAsync("vector-1707268868553", q, limit: 2, minRelevanceScore: 0.8).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}

Error: Azure.RequestFailedException: Unknown field 'Embedding' in vector field list.
Status: 400 (Bad Request)
ErrorCode: InvalidRequestParameter

Content:
{"error":{"code":"InvalidRequestParameter","message":"Unknown field 'Embedding' in vector field list.","details":[{"code":"UnknownField","message":"Unknown field 'Embedding' in vector field list."}]}}

Headers:
Cache-Control: no-cache,no-store
Pragma: no-cache
Server: Microsoft-IIS/10.0
client-request-id: 29f11084-28a5-4b7e-a632-73c88daba9d4
x-ms-client-request-id: 29f11084-28a5-4b7e-a632-73c88daba9d4
request-id: 29f11084-28a5-4b7e-a632-73c88daba9d4
elapsed-time: 38
Strict-Transport-Security: REDACTED
Date: Wed, 07 Feb 2024 09:24:13 GMT
Content-Length: 200
Content-Type: application/json; charset=utf-8
Content-Language: REDACTED
Expires: -1

   at Azure.Search.Documents.SearchClient.SearchInternal[T](SearchOptions options, String operationName, Boolean async, CancellationToken cancellationToken)
   at Azure.Sea